# Process Design Example: Chlorination <a class="anchor" id="top"></a>

- **Prepared by:**
    
    - [Yalin Li](https://qsdsan.readthedocs.io/en/latest/authors/Yalin_Li.html)
    - [Philipp Steiner](https://www.eawag.ch/en/aboutus/portrait/organisation/staff/profile/philipp-steiner/show/)
    - [Eva Reynaert](https://www.eawag.ch/en/aboutus/portrait/organisation/staff/profile/eva-reynaert/show/)

- **Covered topics:**

    - [1. Design Algorithms](#s1)
    - [2. Process Algorithms](#s2)
    - [3. Unit Classes](#s3)
    - [4. System, TEA, and LCA](#s4)

---
### Note
This tutorial is under active development.

---

In [1]:
# Add the path to your cloned repos
import os, sys
coding_path = os.path.abspath(os.path.join(sys.path[0], '../../../../'))
for abbr in ('tmo', 'bst', 'qs'):
    sys.path.append(os.path.join(coding_path, abbr))

In [2]:
import qsdsan as qs
print(f'This tutorial was made with qsdsan v{qs.__version__}.')

This tutorial was made with qsdsan vNone.


### Summary
In this example, we will show how we can set up a chlorination process in `QSDsan`, which would include a contact zone, mixing/storage tanks for the chemical sodium hypochlorite (NaOCl) and treated water, and pumps (contact zone, NaOCl dosing, water storage).

The algorithms largely follows the methodoly presented in  Jones et al., Life cycle environmental impacts of disinfection technologies used in small drinking water systems. *Environmental Science & Technology*, **2018**, *52* (5), 2998-3007. https://doi.org/10.1021/acs.est.7b04448

## 1. Design Algorithms <a class="anchor" id="s1"></a>

### 1.1. Contact zone

In the contact zone, chlorine (in the form of NaOCl) is added and reacts with the influent stream to inactivate microorganims (e.g., viruses, bacteria, protoza). In this example, the contact zone is modeled as a serpentine tubing.

To determine the amount of NaOCl to be added, we will need to calculate the CT (concentration$*$time) values required by the inactivation target.

Let's assume that we will use the following table from U.S. Environmental Protection Agency to determine the CT (in min-mg/L) for 4-log inactivation of viruses by free chlorine (Table B-2 on Page B-3 in this [Disinfection Profiling and Benchmarking Technical Guidance Manual](https://www.epa.gov/system/files/documents/2022-02/disprof_bench_3rules_final_508.pdf)).

| Temperature (°C) | pH=6-9 | pH=10 |
|       :-:        |   :-:  |  :-:  |
|       0.5        |   12   |   90  |
|         5        |    8   |   60  |
|        10        |    6   |   45  |
|        15        |    4   |   30  |
|        20        |    3   |   22  |
|        25        |    2   |   15  |

With the CT value, the desired contact time $T_{contact}$ can be calculated from the desired residual chlorine concentration $C_{res}$ (see Section 2.1): 
$$
T_{contact}[min] = \frac{CT [\frac{mg*min}{L}]}{C_{res}[\frac{mg}{L}]}
$$
    
To get the required detention time $T_{DT}$, the desired contact time needs to be corrected by a baffling factor (BF) that accounts for potential short-circuiting:

$$
T_{DT} [min] = \frac{T_{contact}}{BF} 
$$

A BF value of 0.7 is typical for the serpentine tubing configuration.

Dimensions of the serpentine tubing can then be calculated from the $T_{DT}$:

$$
T_{DT} [min] = \frac{L_p}{v} = L_p * \frac{\pi*{(\frac{d_p}{2})^2}}{Q} = (AS*d_p) * \frac{\pi*{(\frac{d_p}{2})^2}}{Q}
$$

where:
- $L_p$ and $d_p$ are the length and diameter of the pipe (both in m), respectively
- AS is the aspect ratio as in $\frac{L_p}{d_p}$, recommended to be ≧160 by the Colorado Department of Public Health and Environment as in page 16 of this [Baffling Factor Guidance Manual](https://www.colorado.gov/pacific/sites/default/files/CDPHE%20Baffling%20Factor%20Guidance%20Manual.pdf)
- Q and v are the volumetric flow rate and velocity of the influent stream, respectively

Solve for $d_p$:

$$
d_p [m] = (\frac{4T_{DT}*Q}{\pi*AS})^{1/3}
$$

Then we can calculate the amount of material needed:

$$
V_{PVC} [m^3] = \pi * L_p * ((\frac{d_p}{2}+t_{pipe})^2 - d_p^2)
$$

where $t_{pipe}$ is the thickness of the pipe.

### 1.2. Chlorine tank

A cylindrical tank will be used for the storage of the NaOCl solution. For a certain refill inteval $t_{refill}$, volume of the storage tank for 15 wt% NaOCl solution will be:
    
$$
V_{NaOCl_{sol}}[m^3] = \frac{M_{Cl_2}[\frac{kg}{hr}]*t_{refill}[day]*24[\frac{hr}{day}]*\frac{MW_{NaOCl}}{MW_{Cl_2}}}{0.15*\rho_{sol}[\frac{kg}{m^3}]}
$$
 
where:
- $M_{Cl_2}$ is the mass flowrate of $Cl_2$ (can be calculated from $C_{res}$, refer to Section 2.1.)
- $MW_{Cl_2}$ and $MW_{NaOCl}$ are the molar mass of $Cl_2$ (70.91 $\frac{g}{mol}$) and NaOCl (74.44 $\frac{g}{mol}$), respectively
- $\rho_{sol}$ is the density of a 15% NaOCl solution (1200 $\frac{kg}{m^3}$)

Given that

$$
V_{NaOCl_{sol}} = \frac{\pi}{4}d_{cyl}^2*h_{cyl} = \frac{\pi}{4}d_{cyl}^2*AS*d_{cyl} = \frac{\pi}{2}d_{cyl}^3
$$

The diameter of the cylinder tank needed to hold this volume is:
    
$$
d_{cyl} = \sqrt[3]{\frac{2*V_{NaOCl_{sol}}}{\pi}}
$$

The corresponding PVC volume is:

$$
V_{wall} = \pi*h_{cyl}*((d_{cyl}+2*t_{cyl})^2-d_{cyl}^2) = \pi*AS*d_{cyl}*((d_{cyl}+2*t_{cyl})^2-d_{cyl}^2)
$$

$$
V_{floor} = \pi*(d_{cyl}+2*t_{cyl})^2*t_{cyl}
$$

$$
V_{PVC} [m^3] = V_{wall}+V_{floor}
$$

where:
- $h_{cyl}$, $d_{cyl}$, and $t_{cyl}$ are the height, inner diameter, and wall thickness of the cylindrical tank (all in m), respectively
- AS is the aspect ratio as in $\frac{h_cyl}{d_cyl}$

### 1.3. Pumps

For the design of the pumps, we will use the general algorithms in the `WWTpump` class in `QSDsan` (despite of the name, the pump algorithms are not limited to wastewater treatment settings).

## 2. Process Algorithms <a class="anchor" id="s2"></a>

### 2.1. Chlorine dose

Based on the following equation to take into account the amount of chlorine lost to reactions with organics (quantified as the total organic carbon, TOC and ultraviolet absorbance, UVA), we can back-calculate $C_0$ using $C_{res}$:

$$
C_{res} = -0.8404C_0*ln\frac{C_0}{C_{res}} - 0.404TOC [\frac{mg_{C}}{L}]*T_{contact}*(\frac{C_0}{UVA [1/cm]})^{-0.9108} + C_0
$$

<font color='red'>
    
- <strong>Q1:</strong> I'm not sure how TOC and UVA are quantified (e.g., units for them in the equation above)?  
- <strong>Q2:</strong> Are there two solutions of $C_0$ at a certain $C_{res}$? If so, we probably would want to use the lower value.
    - We might want to double-check the results get from ``scipy`` vs. ``flexsolve``
    
</font>

With $C_0$ solved, we will know how much NaOCl we need to add to achieve the desired CT:

$$
M_{Cl_2}[\frac{kg}{hr}] = Q[\frac{m^3}{hr}]*C_0[\frac{g}{m^3}]*\frac{1[kg]}{1000[g]}
$$

where:
- $M_{Cl_2}$ is the mass flowrate of $Cl_2$
- $Q$ is the volumetric flowrate of the influent

### 2.2. Pumping energy

Pumping energy can be calculated based on the flow rate and head pressure/loss as:

$$
P [kW] = \frac{mgH}{1000\eta}
$$

where:
- $m$ is mass flow rate in $[\frac{kg}{s}]$
- $H$ is the head pressure/loss $[m]$
- $\eta$ is the typical pump efficiency (set to 60%)

#### 2.2.1. For the contact zone

In the case of serpentine tubing, head loss is the sum of the major head loss ($H_f$; due to friction) and minor head loss ($H_m$; due to bends in flow):

$$
H [m] = H_f + H_m
$$

For the major head loss, the [Hazen-Williams equation](https://en.wikipedia.org/wiki/Hazen%E2%80%93Williams_equation) can be used (coefficients from [here](https://www.engineeringtoolbox.com/hazen-williams-water-d_797.html)):

$$
H_f = \frac{0.2083*(\frac{100*Q}{C})^{1.852}}{100*d_p^{4.8655}} * L_p
$$

where C is the roughness coefficient and assumed to be 150 for PVC.

The minor head loss can be calculated as:

$$
H_m = \frac{\epsilon*v^2}{2g} * N_{bend}
$$

where:
- $\epsilon$ is the minor loss coefficient and assumed to be 1.5
- $N_{bend}$ is the number of bends can be calculated by dividing the total length by the segment length

$N_{bend}$ can be calculated as

$$
N_{bend} = \frac{L_p}{L_{seg}}
$$

and the segment length $L_{seg}$ can be calculated based on the segment length-to-diameter ratio (recommended to be ≦40 by the Colorado Department of Public Health and Environment as in page 15 of this [Baffling Factor Guidance Manual](https://www.colorado.gov/pacific/sites/default/files/CDPHE%20Baffling%20Factor%20Guidance%20Manual.pdf)).

#### 2.2.2. For the storage tank

For the cylindrical storage tank, there is no minor head loss, therefore the total head loss only comes from the friction loss. However, we need to consider head pressure needed for clorine addition, which is assumed to be 70.3 m. Therefore, the total head needed is

$$
H = H_f + H_p = \frac{0.2083*(\frac{100*Q}{C})^{1.852}}{100*d_{cyl}^{4.8655}} * h_{cyl} + 70.3
$$

[Back to top](#top)

## 3. Unit Classes <a class="anchor" id="#s3"></a>

### 3.1. Contact zone

For the contact zone, we need to create a new class. Check out the tutorials on `SanUnit` ([basic](https://qsdsan.readthedocs.io/en/latest/tutorials/4_SanUnit_basic.html), [advanced](https://qsdsan.readthedocs.io/en/latest/tutorials/5_SanUnit_advanced.html)) for how to make a new `SanUnit` subclass.

In [3]:
from warnings import warn
from math import log, pi, ceil
from flexsolve import IQ_interpolation
from qsdsan import SanUnit, Construction
from qsdsan.sanunits import WWTpump

class ContactZone(SanUnit):
    '''
    Contact zone for water disinfection using chlorine (in the form of sodium hypochlorite, NaOCl).

    Parameters
    ----------
    ins : Iterable(obj)
        Influent stream, NaOCl (updated upon unit simulation).
    outs : obj
        Disinfected stream.
    target_CT : float
        Desired CT (concentration*time) for microorganism in min-mg/L.
    C_res : float
        Desired residual concentration of disinfectant in mg/L.
    UVA : float
        Disinfection credit from UVA.
    PVC_thickness : float
        Thickness of the PVC material in m.

    References
    ----------
    [1] Jones et al., Life cycle environmental impacts of disinfection technologies
    used in small drinking water systems.
    Environmental Science & Technology, 2018, 52 (5), 2998-3007.
    https://doi.org/10.1021/acs.est.7b04448
    [2] Disinfection Profiling and Benchmarking Technical Guidance Manual.
    U.S. Environmental Protection Agency.
    https://www.epa.gov/system/files/documents/2022-02/disprof_bench_3rules_final_508.pdf
    [3] Baffling Factor Guidance Manual.
    Colorado Department of Public Health and Environment.
    https://www.colorado.gov/pacific/sites/default/files/CDPHE%20Baffling%20Factor%20Guidance%20Manual.pdf

    Examples
    --------
    Here we will skip this as we will show how to use it later.
    '''

    _N_ins = 2 # influent stream, NaOCl solution
    _N_outs = 1 # disinfected water
    baffling_factor = 0.7
    aspect_ratio = 160 # length over diamteter
    segment_L_to_dia = 40 # segment length to diameter ratio
    C = 150 # roughness coefficient
    epsilon = 1.5 # minor loss coefficient
    pump_eff = 0.6

    def __init__(self, ID='', ins=None, outs=(), thermo=None, init_with='WasteStream',
                 target_CT=4, # based on the table, set default at T=15°C and pH=6-9
                 C_res=10, UVA=1, #!!! need to update
                 PVC_thickness=0.005,
                 **kwargs):
        SanUnit.__init__(self, ID, ins, outs, thermo, init_with)
        self.target_CT = target_CT
        self.C_res= C_res
        self.UVA = UVA
        self.PVC_thickness = PVC_thickness
        for attr, val in kwargs: setattr(self, kwargs)

        # To consider LCA impacts from the construction material
        self.construction = (
            Construction('ContactZone_PVC', linked_unit=self,
                         item='PVC', quantity_unit='kg'),
            Construction('ContactZone_SS', linked_unit=self,
                         item='StainlessSteel', quantity_unit='kg'),
        )

        # Pump
        ID = self.ID
        eff = self.outs[0]
        self.pump = WWTpump(
            ID=ID+'_pump', ins=eff.proxy(eff.ID+'_proxy'),
            pump_type='', # use the generic pump algorithm
            N_pump=1, capacity_factor=1, include_pump_cost=True,
            include_building_cost=False, include_OM_cost=False,
        )

    # Target function to solve C_0 -->
    #i first thought the found result was not matching another solver,
    # however the equation just has 2 solutions.
    # We'll probably have to discuss with Eva, which solution to use.
    @staticmethod
    def _C_res_at_C_0(C_0, TOC, contact_time, UVA, C_res):
        C_res2 = -0.8404*C_0*log(C_0/C_res) - 0.404*TOC*contact_time*(C_0/UVA)**(-0.9108) + C_0
        return C_res2-C_res

    # Implement process algorithms
    def _run(self):
        inf, naocl = self.ins
        eff, = self.outs

        # Calculate contact time and C_0
        TOC = inf.TOC # in mg/L
        UVA = self.UVA
        C_res = self.C_res
        contact_time = self.target_CT / self.C_res
        try:
            C_0 = IQ_interpolation( # in mg/L
                f=self._C_res_at_C_0, x0=C_res, x1=100*C_res, # assume that C_0 won't be >100X of C_res
                ytol=1e-6, args=(TOC, contact_time, UVA, C_res),
                checkbounds=False)
        except:
            warn('Could not find C_0 for the specified values of TOC, contact_time, UVA and C_res.'
                 'C_0 is assumed to be the same as C_res, resullts may be faulty!')
            C_0 = C_res # assumed

        C_naocl = C_0/70.91*74.44 # 1-to-1 molar conversion of C_0 (for Cl2) to NaOCl
        naocl.imass['NaOCl'] = m_naocl = inf.F_vol * C_naocl / 1000 # m3*mg/L/1000 = kg
        naocl.imass['Water'] = m_naocl/0.15 - m_naocl

        eff.mix_from(self.ins)
        eff.imass['NaOCl'] *= C_res/C_0 # account for the consumed NaOCl

    _units = { # units of measure for the design parameters
        'Pipe diameter': 'm',
        'Pipe length': 'm',
        'Total PVC': 'm3',
        'Pump head': 'm',
        'Pump stainless steel': 'kg',
    }

    # Implement design algorithms
    def _design(self):
        D = self.design_results

        # Pipe dimensions
        contact_time=self.target_CT / self.C_res
        t_DT = contact_time / self.baffling_factor # theoretical detention time
        Q = self.F_vol_in # m3/hr
        t_PVC, AS, C = self.PVC_thickness, self.aspect_ratio, self.C
        dia = (4*t_DT*Q/(pi*AS))**(1/3)
        dia_out = dia + 2*t_PVC
        D['Pipe diameter'] = dia
        L_p = D['Pipe length'] = dia * AS
        V_PVC = D['Total PVC'] = pi * L_p * ((dia_out/2)**2-dia**2)

        # Pump head
        H_f = 0.2083*(100*Q/C)**1.852/(100*dia**4.8655)*L_p # m
        v = Q/(pi*dia**2)
        N_bend = ceil(L_p/(dia*self.segment_L_to_dia))
        H_m = self.epsilon*v**2/(2*9.81)*N_bend
        H = D['Pump head'] = H_f + H_m

        # Pump
        pump = self.pump
        pump.simulate()
        m_ss = D['Pump stainless steel'] = pump.design_results['Pump stainless steel']
        self.power_utility.rate = self.F_mass_in*9.81*H/(1000*self.pump_eff)

        #!!! Will need CAPEX/OPEX/impacts of the UVA lights as well

        # Construction materials for TEA/LCA
        self.construction[0].quantity = V_PVC
        self.construction[1].quantity = m_ss
        self.add_construction(add_cost=True) # this will add PVC and SS cost

    _F_BM_default = {
        'PVC': 1,
        'StainlessSteel': 1,
        'Pump': 1.18*(1+0.007/100),
    }
    def _cost(self):
        C = self.baseline_purchase_costs
        C['Pump'] = self.pump.baseline_purchase_costs['Pump']

### 3.2. ChlorineTank

In [4]:
from qsdsan.sanunits import MixTank

class ChlorineTank(MixTank):
    '''
    A subclass of `MixTank` with an auxiliary pump for chlorine storage.

    Parameters
    ----------
    ins : Iterable(obj)
        NaOCl, water.
    outs : obj
        NaOCl solution.
    t_refill : float
        Tank refill interval in d.
    head_pressure : float
        Assumed head pressure for the pump in m.
    PVC_thickness : float
        Thickness of the PVC material in m.

    See Also
    --------
    `qsdsan.sanunits.MixTank <https://qsdsan.readthedocs.io/en/latest/sanunits/tanks.html#mixtank>`_
    '''

    aspect_ratio = 2 # height over diameter
    C = 150 # roughness coefficient
    pump_eff = 0.6

    def __init__(self, ID='', ins=None, outs=(), thermo=None, init_with='WasteStream',
                 t_refill=7, head_pressure=70.3, PVC_thickness=0.02, **kwargs):
        MixTank.__init__(self, ID, ins, outs, thermo)
        self.head_pressure = head_pressure
        self.PVC_thickness = PVC_thickness
        for attr, val in kwargs: setattr(self, kwargs)

        # To consider LCA impacts from the construction material
        self.construction = (
            Construction('ChlorineTank_PVC', linked_unit=self,
                         item='PVC', quantity_unit='kg'),
            Construction('ChlorineTank_SS', linked_unit=self,
                         item='StainlessSteel', quantity_unit='kg'),
        )
        eff = self.outs[0]
        self.pump = WWTpump(
            ID=self.ID+'_pump', ins=eff.proxy(eff.ID+'_proxy'),
            pump_type='', # use the generic pump algorithm
            N_pump=1, capacity_factor=1, include_pump_cost=True,
            include_building_cost=False, include_OM_cost=False,
        )

    def _run(self):
        naocl, water = self.ins # NaOCl dose will be adjusted when assesmbling the system
        eff = self.outs[0]
        naocl.copy_flow(eff, IDs=('NaOCl',))
        water.copy_flow(eff, IDs=('Water',))


    _units = { # units of measure for the design parameters
        'Tank diameter': 'm',
        'Tank height': 'm',
        'Total PVC': 'm3',
        'Pump head': 'm',
        'Pump stainless steel': 'kg',
    }
    def _design(self):
        MixTank._design(self)
        D = self.design_results
        eff = self.outs[0]

        # Cylindrical tank
        V_naocl = self.ins[0].F_vol #!!! the simulated density is ~1.1 g/mL, want to use 1.2?
        AS, t_PVC = self.aspect_ratio, self.PVC_thickness
        dia = 2*((V_naocl/(pi*AS))**(1/3))
        dia_out = dia + 2*self.PVC_thickness

        D['Tank diameter'] = dia
        h_cyl = D['Tank height'] = dia * AS
        V_wall = pi*h_cyl*(dia_out**2-dia**2)
        V_floor = pi * dia_out**2 * t_PVC
        V_PVC = D['Total PVC'] = V_wall + V_floor

        # Pump
        Q = eff.F_vol
        C = self.C
        pump = self.pump
        H_f = 0.2083*(100*Q/C)**1.852/(100*dia**4.8655) * h_cyl # m
        H_p = self.head_pressure
        D['Pump head'] = H_f + H_p
        pump.simulate()
        m_ss = D['Pump stainless steel'] = pump.design_results['Pump stainless steel']

        # # This is if want to use the default algorithms for calculating electricity usage,
        # # it's more conservative (i.e., the efficiency is lower)
        # pump._H_f = H_f * 3.28 # ft
        # pump._H_p = H_p * 3.28 # ft

        # Construction materials for TEA/LCA
        self.construction[0].quantity = V_PVC
        self.construction[1].quantity = m_ss
        self.add_construction(add_cost=True) # this will add PVC and SS cost

    _F_BM_default = {
        'PVC': 1,
        'StainlessSteel': 1,
        'Pump': 1.18*(1+0.007/100),
    }
    def _cost(self):
        MixTank._cost(self) #!!! this will also add the cost for a stainless steel tank
        pump = self.pump
        self.baseline_purchase_costs['Pump'] = pump.baseline_purchase_costs['Pump']
        H = self.design_results['Pump head']
        self.power_utility.rate += self.F_mass_in*9.81*H/(1000*self.pump_eff)

<font color='red'>
    
- <strong>Q3:</strong> Density of the simulated NaOCl solution is ~1.1 g/mL instead of 1.2, which will make the design more conservative (since volume is larger), do we want to stick to the 1.2?
    - Related, when calculating tank volume, we typically considers a "working volume" factor (<1, our default is 0.8) since we don't want to fill the tank 100% full. So the actual volume will be $\frac{V_{calculated}}{factor}$, do we want to do the same for this storage tank?
    
</font>

## 4. System, TEA, and LCA <a class="anchor" id="#s4"></a>

Finally it's time to create and simulate the entire system.

In [5]:
# Identify the components needed for simulation
import qsdsan as qs
from qsdsan import Component, Components, set_thermo, WasteStream, \
    System, SimpleTEA, ImpactIndicator, ImpactItem, StreamImpactItem, LCA

# Set up components to be used in simulation
kwargs = {
    'phase': 'l',
    'particle_size': 'Soluble',
    'degradability': 'Undegradable',
    'organic': False,
}
H2O = Component('H2O', **kwargs)

kwargs['phase'] = 's'
kwargs['particle_size'] = 'Particulate'
NaOCl = Component('NaOCl', **kwargs)
NaOCl.copy_models_from(qs.Component('HOCl', **kwargs), ['V']) # this gives a rho of ~1.1 g/mL for 15 wt% solution

cmps = Components([H2O, NaOCl])
cmps.compile()
cmps.set_alias('H2O', 'Water')
set_thermo(cmps)

# # Redundant codes, remove after module done
# HCl = Component('HCl', **kwargs)
# HOCl = Component('HOCl', **kwargs)
# NH3 = Component('NH3', **kwargs) # assumed to be liquefied NH3
# cmps = Components([H2O, NaOCl, HCl, HOCl, NH3])
# cmps.set_alias('NH3', 'Ammonia')
# s = WasteStream(Water=85, NaOCl=15, units='kg/hr')


# Impact items for LCA, values all made-up now
GWP = ImpactIndicator('GWP', unit='kg CO2')
PVC = ImpactItem('PVC', GWP=1, price=1)
StainlessSteel = ImpactItem('StainlessSteel', GWP=5, price=5)
NaOCl_item = StreamImpactItem('naocl_item', GWP=2)
e_item = ImpactItem('e_item', functional_unit='kWh', GWP=1.1)

# Streams
influent = WasteStream('influent', Water=100, units='kg/hr') # an assumed fake stream
naocl = WasteStream('naocl', price=1, stream_impact_item=NaOCl_item, units='kg/hr') # price is made-up
water = WasteStream('water')
disinfected = WasteStream('disinfected')

U1 = ContactZone('U1', ins=(influent, 'naocl_solution'), outs=disinfected)
U2 = ChlorineTank('U2', ins=(naocl, water), outs=1-U1)

sys = System('sys', path=(U1, U2))
sys.simulate()

tea = SimpleTEA(sys, discount_rate=0.5, income_tax=0.3, lifetime=10)

get_e_item_quantity = lambda: (sys.get_electricity_consumption()-sys.get_electricity_production())*tea.lifetime
lca = LCA(sys, lifetime=tea.lifetime, e_item=get_e_item_quantity)

/Users/yalinli_cabbi/Library/CloudStorage/OneDrive-Personal/Coding/bst/biosteam/_unit.py:635: RuntimeWarning: the purchase cost item, 'Tanks', has no defined bare-module factor in the 'ChlorineTank.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(warning)


In [6]:
# TEA results
def get_price():
    price = tea.solve_price(disinfected)
    price = price*disinfected.F_mass/disinfected.F_vol # per m3
    return price

get_price()

450.8329582699148

In [7]:
# LCA results
def get_impact():
    impact = lca.get_total_impacts(time=1)['GWP'] # per hour
    impact = impact/disinfected.F_vol # per m3
    return impact

get_impact()

37102.10326916433

<font color='red'>
    
We need the following data for TEA/LCA (below are ones I can think of now, there might be more)
- Lifetime of the equipment and TEA/LCA
- TEA
    - Costs of the unit. If we don't have the cost for the entire unit, we can calculate based on the materials (and we would need the unit costs of PVC/stainless steel), as well as the UVA lights
    - Costs of NaOCl (pure vs. solution?)
    - Electricity usage of UVA lights
    - Other assumptions like discount rate, income tax, etc.
- LCA
    - Life cycle inventory assessment method (e.g., ReCiPe) with the corresponding characterization factors for materials (PVC, stainless steel, UVA lights), chemicals (NaOCl), and electricity. I only used GWP here as an example, we can do any number of LCIA methods/indicators you like.
    
</font>

[Back to top](#top)